In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
from datetime import date
from datetime import datetime
import dtale

In [27]:
dental = pd.read_csv('../data/Smileon.2021.10.17.csv', encoding = "latin-1", skiprows=2)
dental.shape

(57860, 184)

In [28]:
dental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57860 entries, 0 to 57859
Columns: 184 entries, CallReportNum to SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
dtypes: float64(103), int64(6), object(75)
memory usage: 81.2+ MB


In [29]:
#replace spaces with underscores to be able to read  column names
dental.columns = dental.columns.str.replace(' ','_')

In [30]:
list(dental.columns.values)

['CallReportNum',
 'ReportVersion',
 'LinkedToCallReportNum',
 'CallDateAndTimeStart',
 'CallDateAndTimeEnd',
 'CallLength',
 'CallerNum',
 'PhoneWorkerNum',
 'WasRealCall',
 'WasHangup',
 'WasSexCall',
 'WasWrongNumber',
 'WasPrankCall',
 'WasSilentCall',
 'GeoCode',
 'GeoAssignment',
 'CityName',
 'CountyName',
 'StateProvince',
 'CountryName',
 'PostalCode',
 'CensusDivision',
 'CensusTrack',
 'CensusReportingArea',
 '211Region',
 'PhoneType',
 'ThirdPartyName',
 'ThirdPartyOrganization',
 'ThirdPartyPhoneNumber',
 'ThirdPartyAddress',
 'ThirdPartyCity',
 'ThirdPartyCounty',
 'ThirdPartyStateProvince',
 'ThirdPartyPostalCode',
 'Narrative',
 'VolunteerComments',
 'Feedback',
 'CallersFeedback',
 'TextField2',
 'TextField3',
 'TextField4',
 'TextField5',
 'TextField6',
 'TextField7',
 'TextField8',
 'TextField9',
 'TextField10',
 'EnteredByWorkerNum',
 'EnteredOn',
 'Supervisor',
 'Reviewed',
 'FeedbackStatus',
 'FeedbackFromPhoneWorkerNum',
 'FeedbackFromPhoneWorkerName',
 'Referral

In [31]:
dental.drop(['PhoneType', 'OrgNum', 'LinkedToCallReportNum'], axis=1)

,CallReportNum,ReportVersion,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,WasRealCall,WasHangup,WasSexCall,...,SMILE_ON_60+_Screening_-_Treatment_Urgency,SMILE_ON_60+_Screening_-_Untreated_Decay,SMILE_ON_60+_Site_Information_-_Enroller_ID,SMILE_ON_60+_Site_Information_-_Enroller_Location,SMILE_ON_60+_Treatment_Plan_-_Action:_Upload_After_Picture_on_Finish_Tab,SMILE_ON_60+_Treatment_Plan_-_Is_the_treatment_plan_completed?,SMILE_ON_60+_Treatment_Plan_-_Was_a_treatment_plan_developed?,"SMILE_ON_60+_Treatment_Plan_-_Was_the_enrollee_able_to_be_restored_to_function_(can_chew)_and_""social_six""_esthetics_(top_front_six_teeth_are_present_and_disease_free)",SMILE_ON_60+_Verification_of_Eligibility_-_Transportation_Arranged,SMILE_ON_60+_Verification_of_Eligibility_-_Verification_of_Eligibility
0,93115324,SMILE ON 60+ Registration,9/30/2021 16:56,9/30/2021 16:57,1,3638169,123747,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93115188,SMILE ON 60+ Registration,9/30/2021 16:53,9/30/2021 16:53,0,4431194,123747,NaN,NaN,NaN,...,NaN,NaN,NaN,Referral Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN
2,93115081,SMILE ON 60+ Clinical,9/30/2021 16:50,9/30/2021 16:51,1,2941341,161979,NaN,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
3,93115032,SMILE ON 60+ Clinical,9/30/2021 16:48,9/30/2021 16:49,1,3823933,161979,NaN,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
4,93114968,SMILE ON 60+ Clinical,9/30/2021 16:46,9/30/2021 16:48,2,3348256,161979,NaN,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57855,55627514,SMILE ON 60+ Clinical,8/1/2018 8:00,8/1/2018 8:08,8,2385621,118821,NaN,NaN,NaN,...,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57856,67251452,SMILE ON 60+ Clinical,5/31/2018 8:27,5/31/2018 8:29,2,2631417,131347,NaN,NaN,NaN,...,NaN,NaN,Friends in Need Health Center - Kingsport,Provider Site,Not available,No,No,Yes,NaN,NaN
57857,53192452,SMILE ON 60+ Registration,5/23/2018 14:11,5/23/2018 14:11,0,-1,60132,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57858,67915561,SMILE ON 60+ Registration,5/15/2018 9:06,5/15/2018 9:10,4,3239813,128174,NaN,NaN,NaN,...,NaN,NaN,Church Health Center - Memphis,Provider Site,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
dental.dropna(how='all', axis=1, inplace=True)

In [39]:
#duplicates = dental.duplicates(['CallerNum'], keep='first')
newdental = dental.drop_duplicates(subset=['CallerNum'], keep='first')

In [40]:
gender = newdental['SMILE_ON_60+_Demographic_Information_-_Gender'].value_counts()
print(gender)

Female         6774
Male           3815
Transgender       1
Other             1
Name: SMILE_ON_60+_Demographic_Information_-_Gender, dtype: int64


In [41]:
#obtain values in Race/Ethnicity
#column = 'SMILE_ON_60+_Demographic_Information_-_Race/Ethnicity'
RaceEthnicity = newdental['SMILE_ON_60+_Demographic_Information_-_Race/Ethnicity'].value_counts()
print(RaceEthnicity)

Caucasian                            5934
African American                     2660
Hispanic                              456
Asian                                 151
Unknown                               115
American Indian and Alaska Native      52
Multi-racial                           33
Pacific Islander                        9
Name: SMILE_ON_60+_Demographic_Information_-_Race/Ethnicity, dtype: int64


In [42]:
#obtain values of language preferred/needed
#'SMILE_ON_60+_Demographic_Information_-_Preferred_Language/Language_Needed'
language = newdental['SMILE_ON_60+_Demographic_Information_-_Preferred_Language/Language_Needed'].value_counts()
print(language)

English    3154
Spanish     374
Other       116
Arabic       64
Name: SMILE_ON_60+_Demographic_Information_-_Preferred_Language/Language_Needed, dtype: int64


In [43]:
#Other languages
'SMILE_ON_60+_Demographic_Information_-_Other_Language'
otherlanguage = newdental['SMILE_ON_60+_Demographic_Information_-_Other_Language'].value_counts()
print(otherlanguage)

Farsi             4
Mandarin          4
Burmese           3
Amharic           2
Russian           2
Kurdish           2
Vietnamese        2
Laos              2
Kunama            1
Swahili           1
Kinyarwanda       1
Ethiopian         1
Mayan             1
Somali            1
East African      1
Kurdish-Badini    1
Laotian           1
Chinese           1
Nepali            1
Korean            1
English           1
Name: SMILE_ON_60+_Demographic_Information_-_Other_Language, dtype: int64


In [44]:
#obtain Household Income Level
#'SMILE_ON_60+_Demographic_Information_-_Household_Income_Level'
houseincome = newdental['SMILE_ON_60+_Demographic_Information_-_Household_Income_Level'].value_counts()
print(houseincome)

$0 - $21 280     7474
$21246-$24280    1417
$24281-$27315     445
$28806-$32920     398
$32921-$37035     146
$41561-$46755      82
$36366-$41560      73
$43926-$50200      23
$50201-$56475      13
$59046-$67480       9
$58841-$66195       7
$51486-$58840       6
$76121-$85635       3
$84761-$95355       3
$67481-$75915       3
$66606-$76120       2
$74166-$84760       1
Name: SMILE_ON_60+_Demographic_Information_-_Household_Income_Level, dtype: int64


In [45]:
#Enrolled in OTHER assistance programs, which ones?
# 'SMILE_ON_60+_Demographic_Information_-_What_other_assistance_programs_are_you_on?'
otherprograms = newdental['SMILE_ON_60+_Demographic_Information_-_What_other_assistance_programs_are_you_on?'].value_counts()
print(otherprograms)

SSI/SSDI                               2103
SNAP                                    743
TennCare                                448
SNAP; TennCare                          332
SNAP; SSI/SSDI                          295
SNAP; TennCare; SSI/SSDI                284
TennCare; SSI/SSDI                      259
Section 8                               180
SNAP; Section 8                         120
SNAP; TennCare; Section 8                96
SNAP; TennCare; Section 8; SSI/SSDI      92
Section 8; SSI/SSDI                      70
TennCare; Section 8                      51
SNAP; Section 8; SSI/SSDI                46
TennCare; Section 8; SSI/SSDI            41
Name: SMILE_ON_60+_Demographic_Information_-_What_other_assistance_programs_are_you_on?, dtype: int64


In [48]:
#Average of people in a household?
# 'SMILE_ON_60+_Demographic_Information_-_#_of_People_in_Household''
householdnumber = newdental['SMILE_ON_60+_Demographic_Information_-_#_of_People_in_Household'].value_counts()
print(householdnumber)

1         3621
1.0       2812
2         1480
2.0       1244
3          280
3.0        224
4          144
4.0        116
5           81
5.0         72
6.0         34
6           22
7.0         17
7           15
1_____      13
8            9
2_____       9
8.0          6
9            3
3_____       3
9.0          2
0            2
13900        1
8368         1
112          1
1886         1
2864         1
10.0         1
6_____       1
Name: SMILE_ON_60+_Demographic_Information_-_#_of_People_in_Household, dtype: int64


In [54]:
race_list = ['Caucasian']
language_list = ['English']

In [55]:
# we are attempting to filter two columns based on a value
#example: tom_and_42 = df[(df["Name"]=="Tom") & (df["Age"]==42)]

racelanguage = newdental[(newdental["SMILE_ON_60+_Demographic_Information_-_Race/Ethnicity"] == "Caucasian") & (newdental["SMILE_ON_60+_Demographic_Information_-_Preferred_Language/Language_Needed"] == "English")]                      

       CallReportNum              ReportVersion LinkedToCallReportNum  \
1           93115188  SMILE ON 60+ Registration                   NaN   
5           93114880  SMILE ON 60+ Registration                   NaN   
20          93109694  SMILE ON 60+ Registration                   NaN   
47          93096704  SMILE ON 60+ Registration                   NaN   
50          93095080  SMILE ON 60+ Registration                   NaN   
...              ...                        ...                   ...   
56785       56376705  SMILE ON 60+ Registration                   NaN   
56791       56371687  SMILE ON 60+ Registration                   NaN   
56799       56351127  SMILE ON 60+ Registration                   NaN   
56963       56056559  SMILE ON 60+ Registration                   NaN   
57006       56029346  SMILE ON 60+ Registration                   NaN   

      CallDateAndTimeStart CallDateAndTimeEnd  CallLength  CallerNum  \
1          9/30/2021 16:53    9/30/2021 16:53      

In [56]:
d = dtale.show(newdental)
# open it in a new window in browser
d.open_browser()